In [ ]:
import json
import geojson  # to create crop geometry
import gaia

# Todo? gaia.reset()

# Initialize Girder connection
apikey = 'SJoYVdAJdWIockqNdeckt1f7W6JU6Lxou4eYW3ym'
datastore = gaia.connect(apikey=apikey)

In [ ]:
# Create gaia dataset for girder-hosted file
girder_path = 'Public/DEM_bare_earth.tif'  # (~4GB)

# API option 1: create internal url
datastore_url = datastore.lookup_url(girder_path)
dataset = gaia.create(datastore_url)

# API option 2: pass in tuple as the data source
# data_source = (datastore, 'Public/DEM_bare_earth.tif')
# dataset = gaia.create(data_source)

# Todo is there a better api for these options?
dataset.opacity = 0.5
dataset.set_mapnik_style({
    'band': 1,
    'max': 5000,
    'min': 2000,
    'palette': 'matplotlib.Plasma_6',
    'scheme': 'linear',
})

scene = gaia.show(dataset)
display(scene)

In [ ]:
import ipywidgets as widgets
z = widgets.FloatSlider(min=1, max=16, value=scene.zoom, layout=dict(width='95%'))

In [ ]:
newlink = widgets.jslink((z, 'value'), (scene, 'zoom'))
display(z)

In [ ]:
# Generate small crop geometry
meta = dataset.get_metadata()
bounds = meta.get('bounds',{}).get('coordinates')[0]
assert bounds, 'Dataset bounds missing'
# print()
# print(bounds)

# Compute center coordinates
x = (bounds[0][0] + bounds[2][0]) / 2.0
y = (bounds[0][1] + bounds[2][1]) / 2.0

# Use smll percentage of height & width
dx = 0.02 * (bounds[2][0] - bounds[0][0])
dy = 0.02 * (bounds[2][1] - bounds[0][1])
poly = [
    [x,y], [x+dx,y+dy], [x-dx,y+dy], [x-dx,y-dy], [x+dx,y-dy]
]

# Must pass rectangle in as a LIST, in order to get geom formatted the way resgeodata uses
crop_geom = geojson.Polygon([poly])
print()
print(crop_geom)

In [ ]:
# Crop the dataset
from gaia import preprocess
cropped_dataset = preprocess.crop(dataset, crop_geom, name='crop100m.tif')
# Todo output should say "job started" or something like that
# Todo get resonantgeodata to NOT copy source image

In [ ]:
# Todo handle "pending" datasets
print()
cropped_meta = cropped_dataset.get_metadata()
print('Cropped dataset width {}, height {}'.format(
    cropped_meta['width'], cropped_meta['height']))
#print(cropped_meta)

In [ ]:
#scene = gaia.show(cropped_dataset, representation="outline")
cropped_dataset.opacity = 0.9
dataset.opacity= 0.1
scene = gaia.show([cropped_dataset, dataset])
display(scene)

In [ ]:
import ipywidgets as widgets
z = widgets.FloatSlider(min=1, max=16, value=scene.zoom, layout=dict(width='95%'))

In [ ]:
newlink = widgets.jslink((z, 'value'), (scene, 'zoom'))
display(z)

In [ ]:
crop_feature = geojson.Feature(geometry=crop_geom)
fc = geojson.FeatureCollection([crop_feature])
fc

In [ ]:
fc_string = geojson.dumps(fc, sort_keys=True)
#fc_string
filename = 'crop_geometry.geojson'
with open(filename, 'w') as f:
    f.write(fc_string)
